In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Define the search query
location = input("Enter the location: ")
min_price = input("Enter the minimum price: ")
max_price = input("Enter the maximum price: ")
min_area = input("Enter the minimum area: ")
max_area = input("Enter the maximum area: ")
distance = input("Enter the maximum distance from location in km: ")
listing_type = input("Enter listing type (mieszkanie/dom/dzialka): ")
offer_type = input("Enter offer type (sprzedaz/wynajem): ")

# Start the Chrome driver and maximize the window
driver = webdriver.Chrome()
driver.maximize_window()

# Set the variable to check if the banner has been closed
is_banner_closed = False
listings_combined =[]


# Iterate through the pages and open each page in a new window
for page in range(1, 10): #page number
    # Construct the URL
    url =f"https://www.otodom.pl/pl/oferty/{offer_type}/{listing_type}/{location}?priceMin={min_price}&priceMax={max_price}&areaMin={min_area}&areaMax={max_area}&distanceRadius={distance}&page={page}&limit=72"

    # Open the URL in a new window
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(url)

    time.sleep(3)

    # Close Cookies banner
    if not is_banner_closed:
        try:
            cookie_banner = driver.find_element(By.ID,'onetrust-accept-btn-handler')
            cookie_banner.click()
            is_banner_closed = True
        except:
            pass

    # Scroll page down till the end
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Wait for the results to load
    #WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//div[@role='navigation']")))


    # Scroll down to the end of the page
    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(driver.page_source)

    # Find all the relevant information

    listings = soup.find_all("li", {'class': 'css-iq9jxc e1n6ljqa1'})
    listings_combined.append({'page': page, 'listings': listings})

# Close all windows and quit the driver
driver.quit()    

In [31]:
#this gets the Top 13 results only
data = []

for i in listings_combined:
    for record in listings:
    # Extract the link to the record
        link = record.find("a", {'class':'css-1up0y1q e1n6ljqa16'}).get("href")
        link = "https://www.otodom.pl/"+link

    # Extract the title
        property_title = record.find("h3", {'class' :'css-1mmyqk7 e1n6ljqa6'}).get("title")

    # Extract the address
        address =record.find("p", {'class' :'css-14aokuk e1n6ljqa7'}).text.strip()    

    # Extract the number of rooms, area, and price
        details = record.find_all("span", {'class':'css-1ntk0hg ei6hyam1'})
        rooms = ""
        area = ""
        price = ""
        for detail in details:
            if "pokoje" in detail.text:
                rooms = detail.text.strip().replace(" pokoje", "")
            elif "m²" in detail.text:
                area = detail.text.strip().replace(" m²", "")
            elif "zł/mc" in detail.text:
                price = detail.text.strip().replace("zł/mc", "")

        # Append the attributes to the data list
        data.append({
            "Link": link,
            "Property": property_title,
            "Address ": address,
            "Rooms": rooms,
            "Area (m²)": area,
            "Price (zł/mc)": price
        })



# Create a dataframe from the data list
df = pd.DataFrame(data)

In [32]:
df

,Link,Property,Address,Rooms,Area (m²),Price (zł/mc)
0,https://www.otodom.pl//pl/oferta/kawalerka-do-...,Kawalerka do wynajęcia na Warszawskiej Woli,"Warszawa, Wola, al. Prymasa Tysiąclecia",,28,2400
1,https://www.otodom.pl//pl/oferta/zielen-za-okn...,Zieleń za oknem,"Warszawa, Wola, ul. Okopowa",,28.5,2700
2,https://www.otodom.pl//pl/oferta/rezerwacja-do...,"REZERWACJA DO 17.03, Stara Ochota, 2 pokoje","Warszawa, Ochota, Stara Ochota, ul. Grójecka 70",2,40,2600
3,https://www.otodom.pl//pl/oferta/1-pokojowe-um...,"1-pokojowe, umeblowane 31m2 - ul.Kickiego","Warszawa, Praga-Południe, Grochów, ul. Ludwika...",,31,2200
4,https://www.otodom.pl//pl/oferta/atrakcyjne-2-...,Atrakcyjne 2 - pok. mieszkanie RAKOWIEC / OCHOTA,"Warszawa, Ochota",2,50,3000
...,...,...,...,...,...,...
670,https://www.otodom.pl//pl/oferta/perla-na-zoli...,Perła na Żoliborzu!!!,"Warszawa, Żoliborz, Popiełuszki Jerzego ks.",,20,3000
671,https://www.otodom.pl//pl/oferta/wynajme-miesz...,Wynajmę mieszkanie Praga-Południe 40m^2,"Warszawa, Praga-Południe, ul. Rozłucka 16",2,40,2600
672,https://www.otodom.pl//pl/oferta/2-niezalezne-...,2 Niezależne Pokoje - Metro,"Warszawa, Targówek, Krasnobrodzka",2,47,2900
673,https://www.otodom.pl//pl/oferta/luksusowa-kaw...,Luksusowa kawalerka z klimatyzacją + garaż.,"Warszawa, Praga-Południe, Saska Kępa, ul. Arabska",,27,3000


In [36]:
#df["Rooms"] = df["Rooms"].astype(float)
#df["Area (m²)"] = df["Area (m²)"].astype(float)
df["Price (zł/mc)"] = df["Price (zł/mc)"].astype(float)
print(df.dtypes)

Link              object
Property          object
Address           object
Rooms             object
Area (m²)         object
Price (zł/mc)    float64
dtype: object
